In [ ]:
!ln -s "/content/drive/MyDrive/gait" "/content/gait"

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!rm -r /content/Output

In [ ]:
import torch
import cv2
import numpy as np
import torchvision.models.segmentation as segmentation_models
import torchvision
def mask_persons_only(frames, model):
    # Convert the frame to a tensor and normalize it
    preprocess = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    processed_images = []
    for frame in frames:
      input_tensor = preprocess(frame)
      processed_images.append(input_tensor)
    batch_tensor = torch.stack(processed_images)
    batch_tensor = batch_tensor.to(device)
      # Add batch dimension

    # Perform the forward pass to get the output
    with torch.no_grad():
        output = model(batch_tensor)['out']  # The output is a tensor with shape [21, H, W]

    # Get the segmentation map for the "person" class (class 15)
    person_class = 15
    person_masks = output.argmax(1) == person_class
    masked_frames = []
    for i in range(len(person_masks)):
              # Apply the mask to the frame to get the silhouette
              # Convert the mask to uint8
              person_mask = person_masks[i].byte().cpu().numpy()
              person_mask = cv2.resize(person_mask, (frames[i].shape[1], frames[i].shape[0]))
              person_mask = (person_mask * 255).astype(np.uint8)
              masked_frames.append(person_mask)


    return masked_frames

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import torch
import torchvision.models.segmentation as segmentation_models
import imutils
model = segmentation_models.deeplabv3_resnet50(pretrained=True, progress=True, num_classes=21, aux_params=None)
model = model.to(device)
model.eval()
def get_human_silhouettes(video_path, output_dir, batch_size=10, padding=5, threshold=15):
    # Create output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize video capture
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    batch_frames = []
    last_saved_frame = None
    silhouette_list = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB and preprocess
        frame = frame[0: -100, :]
        frame = imutils.resize(frame, width=500)
        batch_frames.append(frame)
        frame_count += 1

        if len(batch_frames) == batch_size or not ret:
            # Perform segmentation on the batch
            predicted_masks = mask_persons_only(batch_frames, model)
            for i in range(len(batch_frames)):
                silhouette = predicted_masks[i]
                contours, _ = cv2.findContours(silhouette, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if len(contours) == 0:
                    continue  # No contours found

                # Find the largest contour
                largest_contour = max(contours, key=cv2.contourArea)
                x, y, w, h = cv2.boundingRect(largest_contour)
                side = max(w, h)
                sidePadding = (side - w) // 2
                topPadding = (side - h) // 2
                cropped = batch_frames[i][y - padding:y + h + padding, x - padding:x + w + padding]
                croppedSillh = silhouette[y - padding:y + h + padding, x - padding:x + w + padding]
                if cropped is None:
                  continue
                # Get contour
                try:
                  gr = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
                except:
                  continue
                _, thresh1 = cv2.threshold(gr, 150, 255, cv2.THRESH_BINARY_INV)
                mergeThresh = cv2.bitwise_and(thresh1, croppedSillh)
                paddedFrame = cv2.copyMakeBorder(mergeThresh, topPadding, topPadding, sidePadding, sidePadding, cv2.BORDER_CONSTANT)

                # Resize to ensure same dimensions
                if last_saved_frame is not None:
                    paddedFrame = cv2.resize(paddedFrame, last_saved_frame.shape[::-1])

                # Save the silhouette frame to the output directory
                if last_saved_frame is None or np.mean(np.abs(paddedFrame.astype(np.float32) - last_saved_frame.astype(np.float32))) >= threshold:
                    last_saved_frame = paddedFrame
                    silhouette_list.append(paddedFrame)

            # Clear the batch lists
            batch_frames = []
            print(f"Processed {frame_count} frames, saved {len(silhouette_list)} frames")

    print(f"Total silhouettes saved: {len(silhouette_list)}")
    cap.release()

    # Compute average image
    if len(silhouette_list) > 0:
        avg_image = silhouette_list[0].astype(np.float32)
        for i in range(1, len(silhouette_list)):
            alpha = 1.0 / (i + 1)
            beta = 1.0 - alpha
            avg_image = cv2.addWeighted(silhouette_list[i].astype(np.float32), alpha, avg_image, beta, 0.0)

        silhouette_path = os.path.join(output_dir, f'silhouette.png')
        cv2.imwrite(silhouette_path, avg_image.astype(np.uint8))
        print(f"Saved final silhouette image to {silhouette_path}")
    else:
        print("No silhouettes to save")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import os
import re

# Define the paths and match string
path = "/content/gait"
output_dir = '/content/gait/Output'
match_string = ".*front\\.mov$"

# Iterate through each person in the directory
for person in os.listdir(path):
    person_path = os.path.join(path, person)

    # Ensure we're only looking at directories
    if os.path.isdir(person_path):
        for file_ in os.listdir(person_path):
            # Check if the file matches the specified pattern
            if re.match(match_string, file_, re.IGNORECASE):
                input_file_path = os.path.join(person_path, file_)

                # Create the output directory for this person if it doesn't exist
                person_output_dir = os.path.join(output_dir, person)
                if not os.path.exists(person_output_dir):
                    os.makedirs(person_output_dir)

                # Call the get_human_silhouettes function
                get_human_silhouettes(input_file_path, person_output_dir)
                print(f"Processed {input_file_path} and saved results to {person_output_dir}")


Total silhouettes saved: 0
No silhouettes to save
Processed /content/gait/sub42/adelka_m_1_front.MOV and saved results to /content/gait/Output/sub42
Total silhouettes saved: 0
No silhouettes to save
Processed /content/gait/sub29/ahmedma_m_1_front.MOV and saved results to /content/gait/Output/sub29
Processed 10 frames, saved 1 frames
Processed 20 frames, saved 1 frames
Processed 30 frames, saved 3 frames
Processed 40 frames, saved 7 frames
Processed 50 frames, saved 11 frames
Processed 60 frames, saved 16 frames
Processed 70 frames, saved 16 frames
Processed 80 frames, saved 17 frames
Processed 90 frames, saved 19 frames
Processed 100 frames, saved 22 frames
Processed 110 frames, saved 25 frames
Processed 120 frames, saved 25 frames
Processed 130 frames, saved 25 frames
Processed 140 frames, saved 25 frames
Processed 150 frames, saved 25 frames
Processed 160 frames, saved 25 frames
Processed 170 frames, saved 25 frames
Processed 180 frames, saved 25 frames
Processed 190 frames, saved 25